# <h1><center id="c1"> 🤹‍♀️ `Prompt` `Engineering` - выжми из модели максимум </center></h1>

In [1]:
import os
from getpass import getpass
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os 

course_api_key = os.environ['LLM_STEPIK_KEY']

In [2]:
# Если используете ключ из курса, запустите эту ячейку
from utils import ChatOpenAI

# Инициализируем языковую модель
llm = ChatOpenAI(temperature=0.0, course_api_key=course_api_key)

Введите API ключ ········


Напишем промпт

In [3]:
prompt = """Ответь на вопрос, опираясь на контекст ниже.
Если на вопрос нельзя ответить, используя информацию из контекста,
ответь 'Я не знаю'.

Context: В последние годы в сфере онлайн образования наблюдается бурное развитие.
Открывается большое количество платформ для хостинга курсов.
Одни из самых крупных платформ в мире, это Coursera и Udemi.
В России лидером является Stepik.

Question: На каких онлайн платформах можно размещать курсы?

Answer: """

In [5]:
print(llm.invoke(prompt))

content='Coursera, Udemi, Stepik.'


## <center id="check1"> 👯‍♀️ Добавление нескольких контекстов </center>

In [2]:
contexts = [
    (
        "Large Language Models (LLMs) are the latest models used in NLP. " +
        "Their superior performance over smaller models has made them incredibly " +
        "useful for developers building NLP enabled applications. These models " +
        "can be accessed via Hugging Face's `transformers` library, via OpenAI " +
        "using the `openai` library, and via Cohere using the `cohere` library."
    ),
    (
        "To use OpenAI's GPT-3 model for completion (generation) tasks, you " +
        "first need to get an API key from " +
        "'https://beta.openai.com/account/api-keys'."
    ),
    (
        "OpenAI's API is accessible via Python using the `openai` library. " +
        "After installing the library with pip you can use it as follows: \n" +
        "```import openai\nopenai.api_key = 'YOUR_API_KEY'\nprompt = \n" +
        "'<YOUR PROMPT>'\nres = openai.Completion.create(engine='text-davinci" +
        "-003', prompt=prompt, max_tokens=100)\nprint(res)"
    )
]

Нам нужно ввести внешнюю информацию в наш промпт между первоначальными инструкциями и пользовательским вводом. Для моделей `OpenAI` рекомендуется отделять контексты от остальной части подсказки с помощью `###` или `"""`, а каждый независимый контекст можно отделить несколькими символами новой строки `\n` и `##`, например:

In [3]:
# Воспользуемся методом join для объединения контекстов в 1 строку с разделителем
context_str = '\n\n##\n\n'.join(contexts)

# Напишем промпт
prompt = f"""Ответь на вопрос, опираясь на контекст ниже.
Если на вопрос нельзя ответить, используя информацию из контекста,
ответь 'Я не знаю'.

###

Contexts:
{context_str}

###

Question: Дай мне 2 примера как использовать GPT-3 на Python

Answer: """

In [30]:
llm = ChatOpenAI(temperature=0.0, course_api_key=course_api_key)
print(llm.invoke(prompt).content)

1. Установите библиотеку `openai` с помощью pip и получите API ключ от OpenAI. Затем используйте функцию `openai.Completion.create()` для создания запроса к GPT-3 модели и получения ответа.
2. Создайте переменную `prompt`, в которой будет содержаться текст, по которому GPT-3 будет генерировать продолжение. Передайте эту переменную в функцию `openai.Completion.create()` в качестве аргумента `prompt`.


# <center id="check2"> 🧐 Как посчитать расходуемые токены и деньги 💸?



Учитывая, что нам может потребоваться ввести внешнюю информацию в наши промпты, они могут стать довольно большими. Максимальное контекстное окно LLM относится к токенам как в промпте, так и в тексте ответа. Для `text-davinci-003` это 4097 токенов. Однако измерение общего количества входных токенов является более сложным.
Поскольку токены не сопоставляются непосредственно со словами, мы можем измерить количество токенов в тексте только путем фактической токенизации текста. Модели GPT используют токенизатор `TikToken` от `OpenAI`. Мы можем установить библиотеку через `Pip`:

Используя токенайзер, можем посчитать количество токенов в предыдущем промпте:

In [4]:
import tiktoken

prompt = f"""Ответь на вопрос, опираясь на контекст ниже.
Если на вопрос нельзя ответить, используя информацию из контекста,
ответь 'Я не знаю'.

###

Contexts:
{context_str}

###

Question: Дай мне 2 примера как использовать GPT-3 на Python

Answer: """

encoder_name = 'p50k_base'
tokenizer = tiktoken.get_encoding(encoder_name)

len(tokenizer.encode(prompt))

440



Это мы получили количество токенов только в промпте, количество токенов для ответа модели регулируется праметром `max_tokens` (по умолчанию 256), указывается максимально возможная длина, т.е. не обязательно, что модель всегда будет расходовать весь доступный лимит токенов, но точно не будет превышать.

In [17]:
llm = ChatOpenAI(temperature=0.0, course_api_key=course_api_key, max_tokens=512)
print(llm.invoke(prompt).content)

1. Установите библиотеку `openai` с помощью pip: `pip install openai`
   Затем импортируйте ее и укажите ваш API-ключ:
   ```python
   import openai
   openai.api_key = 'YOUR_API_KEY'
   ```

2. Создайте запрос к модели GPT-3, используя метод `Completion.create`:
   ```python
   prompt = '<YOUR PROMPT>'
   res = openai.Completion.create(engine='text-davinci-003', prompt=prompt, max_tokens=100)
   print(res)
   ```
   Здесь `prompt` - это текст, который вы хотите передать в модель, а `max_tokens` - это максимальное количество токенов в ответе модели.


In [18]:
len(tokenizer.encode(llm.invoke(prompt)))

1275



Получаем 440 + 504 ~ 1000 токенов на 1 вопрос-ответ, при стоимости 1000 токенов `0.002$`. Т.е. 10 пользователей за 100 запросов сожгут уже `2$`. Причем окно контекста используется только на 1\3. <br>
Начинаем продумывать экономику своего сервиса 🤑